In [3]:
import os

import pandas as pd
import segmentation_models_pytorch as smp
import torch
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader

folder_path = "PTUMOUSE"

seed_value = 23
torch.manual_seed(seed_value)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
def make_csv_files(folder_path, folder):
    images_folder = folder_path + "/" + folder + "images"
    masks_folder = folder_path + "/" + folder + "masks"

    images_files = os.listdir(images_folder)
    masks_files = os.listdir(masks_folder)

    image_paths = [os.path.join(folder + "images", file) for file in images_files]
    mask_paths = [os.path.join(folder + "masks", file) for file in masks_files]

    data = {'orig_image': image_paths, 'mask_image': mask_paths}
    df = pd.DataFrame(data)

    csv_file_path = "train_data.csv" if folder == "" else "test_data.csv"

    df.to_csv(csv_file_path, index=False)

In [5]:
make_csv_files("PTUMOUSE","")
make_csv_files("PTUMOUSE","test_")
train_df = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

### Preprocessing (подготовка данных)

In [6]:
test

,orig_image,mask_image
0,test_images\OFT_control_01$000152&03_@000064.bmp,test_masks\OFT_control_01$000152&03_@000064.bmp
1,test_images\OFT_control_01$000152&03_@004271.bmp,test_masks\OFT_control_01$000152&03_@004271.bmp
2,test_images\OFT_control_02$000146&03_@000152.bmp,test_masks\OFT_control_02$000146&03_@000152.bmp
3,test_images\OFT_control_02$000146&03_@004474.bmp,test_masks\OFT_control_02$000146&03_@004474.bmp
4,test_images\OFT_control_03$000141&03_@000030.bmp,test_masks\OFT_control_03$000141&03_@000030.bmp
5,test_images\OFT_control_03$000141&03_@004498.bmp,test_masks\OFT_control_03$000141&03_@004498.bmp
6,test_images\OFT_control_04$000176&03_@000060.bmp,test_masks\OFT_control_04$000176&03_@000060.bmp
7,test_images\OFT_control_04$000176&03_@004368.bmp,test_masks\OFT_control_04$000176&03_@004368.bmp
8,test_images\OFT_control_05$000124&03_@000055.bmp,test_masks\OFT_control_05$000124&03_@000055.bmp
9,test_images\OFT_control_05$000124&03_@003979.bmp,test_masks\OFT_control_05$000124&03_@003979.bmp


In [7]:
class ImagesDataset(Dataset):
    def __init__(self, folder, data, transform_image, transform_mask):
      self.folder = folder
      self.data = data.copy()
      self.orig_image_paths = [os.path.join(folder, filename) for filename in data['orig_image'].copy()]
      self.mask_image_paths = [os.path.join(folder, filename) for filename in data['mask_image'].copy()]
      self.transform_image = transform_image
      self.transform_mask = transform_mask

    def __len__(self):
        return len(self.orig_image_paths)

    def __getitem__(self, idx):
        orig_image_path = self.orig_image_paths[idx]
        mask_image_path = self.mask_image_paths[idx]
        orig_image = Image.open(orig_image_path).convert('RGB')
        mask_image = Image.open(mask_image_path).convert('L')
        
        orig_image = self.transform_image(orig_image)
        orig_image = orig_image.to(orig_image)
        
        mask_image = self.transform_mask(mask_image)
        mask_image = mask_image.to(mask_image)
        
        return orig_image.float(), mask_image.float()

In [8]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, test_size=0.1 , random_state=42)

size = (320, 544)
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
batch_size = 8

transform_image = transforms.Compose([transforms.Resize(size), transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])

transform_mask = transforms.Compose([transforms.Resize(size), transforms.ToTensor()])

train_dataset = ImagesDataset(folder_path, train, transform_image, transform_mask)
val_dataset = ImagesDataset(folder_path, val, transform_image, transform_mask)
test_dataset = ImagesDataset(folder_path, test, transform_image, transform_mask)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Training/evaluation loop

In [9]:

def iou_score(outputs, labels):
    intersection = torch.logical_and(outputs, labels).sum()
    union = torch.logical_or(outputs, labels).sum()
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.item()

def learning(num_epochs, train_load, val_load, model, optimizer, criterion, model_name, scheduler=None):
    device = next(model.parameters()).device
    train_losses = []
    val_losses = []
    iou_test = []
    max_iou = 0.0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for x_batch, y_batch in tqdm(train_load):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append(train_loss/len(train_load))

        if scheduler is not None:
            scheduler.step(train_loss)

        model.eval()
        val_loss = 0
        with torch.no_grad():
            iou_epoch = 0
            for x_batch, y_batch in tqdm(val_load):
                x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                outputs = model(x_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
                iou = iou_score(outputs, y_batch)
                iou_epoch += iou
            val_losses.append(val_loss/len(val_load))
            iou_test.append(iou_epoch/len(val_load))

            if iou_test[-1] > max_iou:
                max_iou = iou_test[-1]
                torch.save(model.state_dict(), f'{model_name}_best.pth')

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_losses[-1]}, Val Loss: {val_losses[-1]}, IoU: {iou_test[-1]}")

    return model, train_losses, val_losses, iou_test


### Prediction function

### Experiments

In [10]:
model_name = 'efficientnet-b4'
model = smp.UnetPlusPlus(encoder_name=model_name, encoder_weights='imagenet', in_channels=3, classes=1)
model.to(device)

UnetPlusPlus(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          48, 12, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          12, 48, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticS

In [11]:
import segmentation_models_pytorch as smp

learning_rate = 0.001
optimizer = optim.Adamax(model.parameters(), lr=learning_rate)
criterion = smp.losses.DiceLoss(mode='binary')
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

In [12]:
num_epochs = 20
model, train_losses, val_losses, iou_test = learning(num_epochs, train_loader, val_loader, model, optimizer, criterion, model_name, scheduler)

  0%|          | 0/160 [00:00<?, ?it/s]C:\Users\pdd\PycharmProjects\detect\venv\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
C:\Users\pdd\PycharmProjects\detect\venv\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
100%|██████████| 18/18 [00:24<00:00,  1.37s/it]


Epoch 1/20: Train Loss: 0.5213734287768602, Val Loss: 0.11849357022179498, IoU: 0.009873733079681793


100%|██████████| 18/18 [00:30<00:00,  1.68s/it]


Epoch 2/20: Train Loss: 0.0924427542835474, Val Loss: 0.08184040586153667, IoU: 0.009873733079681793


100%|██████████| 18/18 [01:10<00:00,  3.94s/it]


...

Epoch 18/20: Train Loss: 0.04413867443799972, Val Loss: 0.057059897316826716, IoU: 0.009873733079681793


100%|██████████| 18/18 [00:07<00:00,  2.32it/s]


Epoch 19/20: Train Loss: 0.04349689893424511, Val Loss: 0.05920102198918661, IoU: 0.009873733079681793


100%|██████████| 18/18 [00:29<00:00,  1.62s/it]

Epoch 20/20: Train Loss: 0.04343007616698742, Val Loss: 0.06089085340499878, IoU: 0.009873733079681793


In [16]:
model = smp.UnetPlusPlus('efficientnet-b4', classes=1).to(device)
model.load_state_dict(torch.load("gay_sex.pth"))

<All keys matched successfully>

In [17]:
import torch
from torchvision import transforms
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score


def validation(model, loader):
    size = (544, 928)
    model.eval()
    preds1, masks1 = [], []
    tfm = transforms.Resize(size)
    with torch.no_grad():
        for xb, yb in tqdm(loader):
            xb, yb = xb.to('cuda'), yb.to('cuda')
            outs = torch.round(torch.sigmoid(model(xb)))
            outs, yb = map(tfm, [outs, yb])
            preds1.append(outs.cpu().numpy())
            masks1.append(yb.cpu().numpy())
    preds1 = np.concatenate(preds1).squeeze()
    masks1 = np.concatenate(masks1).squeeze()
    result = preds1, masks1
    preds, masks = result
    pred_flat = preds.flatten().astype(int)
    gt_flat = masks.flatten().astype(int)
    precision1 = precision_score(gt_flat, pred_flat)
    recall1 = recall_score(gt_flat, pred_flat)
    result1 = precision1, recall1
    precision, recall = result1
    intersection = np.logical_and(preds, masks).sum()
    union = np.logical_or(preds, masks).sum()
    iou1 = intersection / union if union != 0 else 0
    iou = iou1
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"IoU: {iou:.4f}")


### Evaluation (оценка качества модели)

In [18]:
validation(model, val_loader)

100%|██████████| 18/18 [01:25<00:00,  4.74s/it]


Precision: 0.8367
Recall: 0.9922
IoU: 0.8635


In [19]:
validation(model, test_loader)

100%|██████████| 2/2 [00:06<00:00,  3.04s/it]


Precision: 0.8223
Recall: 0.9762
IoU: 0.8488
